In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import nltk 
import re
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns 
import sklearn.metrics
import sklearn
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Read news aricle csv file

In [ ]:
data = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/ML Freelance/news_articles.csv', encoding="latin", index_col=0)
data = data.dropna()
data.count()

published                  2045
title                      2045
text                       2045
language                   2045
site_url                   2045
main_img_url               2045
type                       2045
label                      2045
title_without_stopwords    2045
text_without_stopwords     2045
hasImage                   2045
dtype: int64

EDA

In [ ]:
data.groupby('label').describe()

hasImage                                             
         count      mean       std  min  25%  50%  75%  max
label                                                      
Fake    1291.0  0.736638  0.440628  0.0  0.0  1.0  1.0  1.0
Real     754.0  0.834218  0.372132  0.0  1.0  1.0  1.0  1.0

In [ ]:
data=data.dropna()

In [ ]:
# Seperating Independent and dependent features
X = data.drop(columns=['label'],axis=1)
y = data['label']

In [ ]:
data.head()

,published,title,text,language,site_url,main_img_url,type,label,title_without_stopwords,text_without_stopwords,hasImage
author,,,,,,,,,,,
Barracuda Brigade,2016-10-26T21:41:00.000+03:00,muslims busted they stole millions in govt ben...,print they should pay all the back all the mon...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,muslims busted stole millions govt benefits,print pay back money plus interest entire fami...,1.0
reasoning with facts,2016-10-29T08:47:11.259+03:00,re why did attorney general loretta lynch plea...,why did attorney general loretta lynch plead t...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,attorney general loretta lynch plead fifth,attorney general loretta lynch plead fifth bar...,1.0
Barracuda Brigade,2016-10-31T01:41:49.479+02:00,breaking weiner cooperating with fbi on hillar...,red state \nfox news sunday reported this mor...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,breaking weiner cooperating fbi hillary email ...,red state fox news sunday reported morning ant...,1.0
Fed Up,2016-11-01T05:22:00.000+02:00,pin drop speech by father of daughter kidnappe...,email kayla mueller was a prisoner and torture...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,bias,Real,pin drop speech father daughter kidnapped kill...,email kayla mueller prisoner tortured isis cha...,1.0
Fed Up,2016-11-01T21:56:00.000+02:00,fantastic trumps point plan to reform healthc...,email healthcare reform to make america great ...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,bias,Real,fantastic trumps point plan reform healthcare ...,email healthcare reform make america great sin...,1.0


In [ ]:
data['text_len'] = data['text'].apply(len)

In [ ]:
data['len_title'] = data['title'].apply(len)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Plot article type distribution
df_type = data['type'].value_counts()
sns.barplot(np.arange(len(df_type)), df_type)
plt.xticks(np.arange(len(df_type)), df_type.index.values.tolist(), rotation=90)
plt.title('Article type count', fontsize=20)
plt.show()

/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


([<matplotlib.axis.XTick at 0x7f3eef5337d0>,
 [Text(0, 0, 'bs'),
  Text(0, 0, 'conspiracy'),
  Text(0, 0, 'bias'),
  Text(0, 0, 'hate'),
  Text(0, 0, 'satire'),
  Text(0, 0, 'state'),
  Text(0, 0, 'junksci'),
  Text(0, 0, 'fake')])

Text(0.5, 1.0, 'Article type count')

In [ ]:
from nltk.corpus import stopwords
import string
stopwords.words('english')[0:10] # Show some stop words

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [ ]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Perform lemmatization
    4. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    lemma = nlp.WordNetLemmatizer()
    nopunc = [ lemma.lemmatize(word) for word in nopunc]

In [ ]:
data['title'].apply(text_process)

author
Barracuda Brigade       [muslims, busted, stole, millions, govt, benef...
reasoning with facts    [attorney, general, loretta, lynch, plead, fifth]
Barracuda Brigade       [breaking, weiner, cooperating, fbi, hillary, ...
Fed Up                  [pin, drop, speech, father, daughter, kidnappe...
Fed Up                  [fantastic, trumps, point, plan, reform, healt...
                                              ...                        
Matt Barber                           [never, trumpers, must, reconsider]
Jane Chastain               [election, crossroads, socialism, capitalism]
Michael Brown                                 [reasons, ill, vote, trump]
Ann Coulter               [new, country, women, minorities, hit, hardest]
Larry Elder                      [trump, vs, clinton, risk, vs, disaster]
Name: title, Length: 2045, dtype: object

In [ ]:
data['text'].head(5).apply(text_process)

author
Barracuda Brigade       [print, pay, back, money, plus, interest, enti...
reasoning with facts    [attorney, general, loretta, lynch, plead, fif...
Barracuda Brigade       [red, state, fox, news, sunday, reported, morn...
Fed Up                  [email, kayla, mueller, prisoner, tortured, is...
Fed Up                  [email, healthcare, reform, make, america, gre...
Name: text, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer(analyzer=text_process).fit(data['text'])

# Print total number of vocab words
print(len(bow_transformer.vocabulary_))

47111


In [ ]:
messages_bow = bow_transformer.transform(data['text'])

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(messages_bow)

messages_tfidf = tfidf_transformer.transform(messages_bow)
print(messages_tfidf.shape)

(2045, 47111)


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

y = le.fit_transform(data.label)

Training & Testing

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, label_train, label_test = train_test_split(data['text'], y, test_size=0.2, random_state = 42)

In [ ]:
print(len(X_train), len(X_test), len(X_train) + len(X_test))

1636 409 2045


# Naive Bayes

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(messages_tfidf, data['label'])

In [ ]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [ ]:
pipeline.fit(X_train,label_train)

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7f3eef489440>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [ ]:
predictions1 = pipeline.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(predictions1,label_test))

              precision    recall  f1-score   support

           0       0.99      0.66      0.79       396
           1       0.08      0.85      0.14        13

    accuracy                           0.67       409
   macro avg       0.53      0.75      0.47       409
weighted avg       0.96      0.67      0.77       409



Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=50, criterion='entropy',random_state=0)
classifier.fit(messages_tfidf, data['label'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
pipeline_rf = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', RandomForestClassifier()),  # train on TF-IDF vectors w/ SVM
])


In [ ]:
pipeline_rf.fit(X_train,label_train)

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7f3eef489440>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.

In [ ]:
predictions2 = pipeline_rf.predict(X_test)

In [ ]:
print(classification_report(predictions2,label_test))

              precision    recall  f1-score   support

           0       0.93      0.78      0.85       315
           1       0.52      0.81      0.64        94

    accuracy                           0.79       409
   macro avg       0.73      0.79      0.74       409
weighted avg       0.84      0.79      0.80       409



DT

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42, criterion="entropy",
                             min_samples_split=10, min_samples_leaf=10, max_depth=3, max_leaf_nodes=5)

In [ ]:
pipeline_dt = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', DecisionTreeClassifier()),  # train on TF-IDF vectors w/ SVM
])

In [ ]:
pipeline_dt.fit(X_train,label_train)

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7f3eef489440>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b...
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features=None, max_leaf_nodes=None,
                                       

In [ ]:
predictions_dt = pipeline_dt.predict(X_test)

In [ ]:
print(classification_report(predictions_dt,label_test))

              precision    recall  f1-score   support

           0       0.81      0.82      0.81       259
           1       0.68      0.66      0.67       150

    accuracy                           0.76       409
   macro avg       0.74      0.74      0.74       409
weighted avg       0.76      0.76      0.76       409



Gradient Boosting

svm

In [ ]:
from sklearn.svm import LinearSVC
svm=LinearSVC()

In [ ]:
pipeline_svm = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', LinearSVC()),  # train on TF-IDF vectors w/ SVM
])

In [ ]:
pipeline_svm.fit(X_train,label_train)

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7f3eef489440>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, i

In [ ]:
predictions_svm = pipeline_gbm.predict(X_test)

In [ ]:
print(classification_report(predictions_svm,label_test))

              precision    recall  f1-score   support

           0       0.93      0.81      0.86       305
           1       0.59      0.83      0.69       104

    accuracy                           0.81       409
   macro avg       0.76      0.82      0.78       409
weighted avg       0.85      0.81      0.82       409

